Most existing approaches to visual odometry are based on the following stages.

1. Acquire input images
2. Image correction: apply image processing techniques for lens distortion removal, etc.
3. Feature detection: define interest operators, and match features across frames and construct optical flow field.
       
4. Check flow field vectors for potential tracking errors and remove outliers.
5. Estimation of the camera motion from the optical flow.
       
6. Periodic repopulation of trackpoints to maintain coverage across the image.

In [ ]:
################################Oxford Dataset################################
%run robotcar-sdk/python/camera_model.py
from skimage.io import imread_collection

#oxford(ox) setup
models_dir = 'robotcar-sdk/models'
ox_img_dir = 'oxford_2014-05-14-13-50-20/stereo/left/'
ox_cam = CameraModel(models_dir,ox_img_dir)
ox_imgs = imread_collection(ox_img_dir+'*.png', conserve_memory = True)
# ox_imgs = np.array(ox_imgs)
print "ox loaded."

In [1]:
################################KITTI Dataset################################

#adapted from https://github.com/uoip/monoVO-python 
%run pinhole_camera.py

##kitti setup
poses_dir = 'dataset/poses/00.txt' #for ground truth
img_dir = 'dataset/sequences/00/image_0/'
cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)
print "kitti loaded."

kitti loaded.


In [ ]:
################################IMAGE CORRECTION/FEATURE DETECTION################################

vo = VisualOdometry(cam, poses_dir)

# ox_vo = VisualOdometry(ox_cam, poses_dir)

traj = np.zeros((600,600,3), dtype=np.uint8)

#drawing trajectories for each frame
for img_id in xrange(10):
    img = cv2.imread(img_dir+str(img_id).zfill(6)+'.png', 0)
    
    vo.update(img, img_id)

    cur_t = vo.cur_t
    if(img_id > 2): 
        x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else: 
        x, y, z = 0., 0., 0.
        
    #offset so the 2 trajectories do not overlap
    x_offset, y_offset = 0, 0
    draw_x, draw_y = int(x)+(290-x_offset), int(z)+(90-y_offset)
    true_x, true_y = int(vo.trueX)+290, int(vo.trueZ)+90

    #predicted trajectory
    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/4540,255-img_id*255/4540,0), 1)
    #actual trajectory
    cv2.circle(traj, (true_x,true_y), 1, (0,0,255), 1)
    
    cv2.rectangle(traj, (10, 20), (600, 60), (0,0,0), -1)
    text = "Coordinates: x=%2fm y=%2fm z=%2fm"%(x,y,z)
    cv2.putText(traj, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)
    frame = "Frame: " + str(img_id)
    cv2.putText(traj, frame, (20,60), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)
    
    cv2.imshow('Road facing camera', img)
    cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)


cv2.imwrite('map.png', traj)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
##########################Adapting Existing Code################################

In [31]:
import cv2
#adapted from https://github.com/avisingh599/mono-vo
def track_features(img1, img2, points1):
    #lucas kanade(lk) optical flow parameters
    lk_params = dict(winSize  = (21, 21), 
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    #open cv method for generating LK optical flow
    #shape: [k,2] [k,1] [k,1]
    points2, status, errs = cv2.calcOpticalFlowPyrLK(img1, img2, points1, None, **lk_params)
    status = status.reshape(status.shape[0])
    
    #removing points for which the tracking has failed/gone outside the frame
    indexCorrection = 0
    for i in range(len(status)):
        pt = points2[i-indexCorrection]
        if(status[0] == 0 or (pt[0]<0 or pt[1]<0)):
            if(pt[0]<0 or pt[1]<0): status[i]=0
                
            np.delete(points1, (i-indexCorrection))
            np.delete(points2, (i-indexCorrection))            
            indexCorrection+=1
            
    return points1[status==1], points2[status==1]    

def detect_features(img1):
    #detector parameters
    threshold_value = 20
    nonMax = True
    
    key_points1 = cv2.FastFeatureDetector_create(
        threshold=threshold_value, 
        nonmaxSuppression=nonMax)
    
    return key_points1.detect(img1)

In [ ]:
# def feature_tracking(img1, img2, kp1):
#     kp2, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, kp1, None, **lk_params)

#     st = st.reshape(st.shape[0])
#     kp1 = kp1[st == 1]
#     kp2 = kp2[st == 1]

#     return kp1, kp2

In [32]:
# %run pinhole_camera.py
# %matplotlib inline
# import matplotlib.pyplot as plt

#first 2 frames
img1 = cv2.imread(img_dir+'000000.png', 0)
img2 = cv2.imread(img_dir+'000001.png', 0)

key_points1 = detect_features(img1)

#x.pt is an interface feature of cv2 Keypoints
#pt stands for coordinates of the keypoint
#source: https://goo.gl/59X81j
key_points1 = np.array([x.pt for x in key_points1], dtype=np.float32)

key_points1, key_points2 = track_features(img1, img2, key_points1)


#Finding the essential matrix
# E = cv2.findEssentialMat(points2, points1, )

In [ ]:
import numpy as np 
import cv2

STAGE_FIRST_FRAME = 0
STAGE_SECOND_FRAME = 1
STAGE_DEFAULT_FRAME = 2
kMinNumFeature = 1500

#lucas kanade(lk) optical flow
lk_params = dict(winSize  = (21, 21), 
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

def featureTracking(image_ref, image_cur, px_ref):
    kp2, st, err = cv2.calcOpticalFlowPyrLK(image_ref, image_cur, px_ref, None, **lk_params)  #shape: [k,2] [k,1] [k,1]

    st = st.reshape(st.shape[0])
    kp1 = px_ref[st == 1]
    kp2 = kp2[st == 1]

    return kp1, kp2


class PinholeCamera:
    """
    Attributes:
    fx, fy: focal lengths expressed as pixel units
    cx, cy: image principle points
    source: https://goo.gl/6LPrcX 
    
    """
    def __init__(self, width, height, fx, fy, cx, cy, 
                k1=0.0, k2=0.0, p1=0.0, p2=0.0, k3=0.0):
        self.width = width
        self.height = height
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.distortion = (abs(k1) > 0.0000001)
        self.d = [k1, k2, p1, p2, k3]


class VisualOdometry:
    def __init__(self, cam, poses):
        self.frame_stage = 0
        self.cam = cam
        self.new_frame, self.last_frame = None, None
        self.cur_R, self.cur_t = None, None
        self.px_ref = None
        self.px_cur = None
        self.focal = cam.fx
        self.pp = (cam.cx, cam.cy)
        self.trueX, self.trueY, self.trueZ = 0, 0, 0
        self.detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
        with open(poses) as f:
            self.poses = f.readlines()

    def getAbsoluteScale(self, frame_id):  #specialized for KITTI odometry dataset
        ss = self.poses[frame_id-1].strip().split()
        x_prev = float(ss[3])
        y_prev = float(ss[7])
        z_prev = float(ss[11])
        ss = self.poses[frame_id].strip().split()
        x = float(ss[3])
        y = float(ss[7])
        z = float(ss[11])
        self.trueX, self.trueY, self.trueZ = x, y, z
        return np.sqrt((x - x_prev)*(x - x_prev) + (y - y_prev)*(y - y_prev) + (z - z_prev)*(z - z_prev))

    def processFirstFrame(self):
        self.px_ref = self.detector.detect(self.new_frame)
        self.px_ref = np.array([x.pt for x in self.px_ref], dtype=np.float32)
        self.frame_stage = STAGE_SECOND_FRAME

    def processSecondFrame(self):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        
        _, self.cur_R, self.cur_t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        
        self.frame_stage = STAGE_DEFAULT_FRAME 
        self.px_ref = self.px_cur

    def processFrame(self, frame_id):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        absolute_scale = self.getAbsoluteScale(frame_id)
        if(absolute_scale > 0.1):
            self.cur_t = self.cur_t + absolute_scale*self.cur_R.dot(t) 
            self.cur_R = R.dot(self.cur_R)
        if(self.px_ref.shape[0] < kMinNumFeature):
            self.px_cur = self.detector.detect(self.new_frame)
            self.px_cur = np.array([x.pt for x in self.px_cur], dtype=np.float32)
        self.px_ref = self.px_cur

    def update(self, img, frame_id):
        assert(img.ndim==2 and img.shape[0]==self.cam.height and img.shape[1]==self.cam.width), "Frame: provided image has not the same size as the camera model or image is not grayscale"
        self.new_frame = img
        if(self.frame_stage == STAGE_DEFAULT_FRAME):
            self.processFrame(frame_id)
        elif(self.frame_stage == STAGE_SECOND_FRAME):
            self.processSecondFrame()
        elif(self.frame_stage == STAGE_FIRST_FRAME):
            self.processFirstFrame()
        self.last_frame = self.new_frame
